# AWS CloudFormation

<img src="../_assets/aws_service_icons/cloudformation.svg" width="80" alt="AWS CloudFormation">

## Goals
- Understand what CloudFormation is
- Know what it’s practically used for
- See a simple **SDK pseudo-code** example (not executed)


## What is CloudFormation?

**AWS CloudFormation** is AWS’s **Infrastructure as Code (IaC)** service.

You describe AWS resources (VPCs, S3 buckets, IAM roles, Lambda functions, etc.) in a **template** (YAML/JSON), and CloudFormation provisions and manages them as a **stack**.

Key idea:
- A template is the **desired state**.
- A stack is the **running instance** of that template.
- CloudFormation computes changes, creates/updates/deletes resources in the right order, and can **roll back** on failure.


## What is CloudFormation used for (practically)?

CloudFormation is used to make infrastructure:
- **Repeatable**: create dev/stage/prod environments from the same template.
- **Reviewable**: templates can live in Git and be code-reviewed.
- **Automatable**: integrate infra changes into CI/CD pipelines.
- **Safer**: preview changes with **Change Sets** and rely on rollback on errors.

Common practical workflows:
- Provisioning baseline networking (VPC, subnets, routing) and shared services.
- Creating application stacks (compute, IAM, storage, queues, alarms).
- Enforcing consistent tagging, security defaults, and compliance controls.
- Spinning up temporary environments for experiments or demos (with cleanup).


## Core concepts (minimum you should know)

- **Template**: YAML/JSON document describing resources.
- **Stack**: a deployment of a template (create/update/delete).
- **Parameters**: inputs to a template (e.g., environment name, instance size).
- **Outputs**: exported values you can read after deployment (e.g., ARNs, endpoints).
- **Change Sets**: preview what will change before applying an update.
- **Capabilities**: acknowledgements required for sensitive operations (notably IAM), e.g. `CAPABILITY_NAMED_IAM`.
- **Drift detection**: detect when resources were changed outside CloudFormation.


## Using CloudFormation with an SDK (pseudo-code)

This is **illustrative pseudo-code** showing a typical flow with an AWS SDK (e.g., Python `boto3`, JS v3, Java, etc.).

**Note**: creating/updating stacks provisions real AWS resources and can incur cost; review changes before deploying.

```python
# PSEUDO-CODE (do not run)

import boto3

region = "us-east-1"
cf = boto3.client("cloudformation", region_name=region)

stack_name = "demo-cloudformation-stack"

# 1) Load a template (YAML/JSON). For larger templates you may prefer TemplateURL (S3).
template_body = open("template.yaml", "r", encoding="utf-8").read()

# 2) Provide parameters declared in the template (optional).
parameters = [
    {"ParameterKey": "Env", "ParameterValue": "dev"},
    {"ParameterKey": "BucketName", "ParameterValue": "my-demo-bucket"},
]

# 3) Create the stack (one-time).
cf.create_stack(
    StackName=stack_name,
    TemplateBody=template_body,
    Parameters=parameters,
    Capabilities=["CAPABILITY_NAMED_IAM"],  # required if the template creates IAM resources
    Tags=[{"Key": "project", "Value": "ckc"}],
)

# 4) Wait for completion, then read stack outputs (e.g., ARNs, endpoints).
cf.get_waiter("stack_create_complete").wait(StackName=stack_name)

resp = cf.describe_stacks(StackName=stack_name)
outputs = resp["Stacks"][0].get("Outputs", [])

# 5) Update the stack when the template changes.
try:
    cf.update_stack(
        StackName=stack_name,
        TemplateBody=template_body,
        Parameters=parameters,
        Capabilities=["CAPABILITY_NAMED_IAM"],
    )
    cf.get_waiter("stack_update_complete").wait(StackName=stack_name)
except cf.exceptions.ClientError as e:
    # Common case: "No updates are to be performed."
    handle_update_error(e)

# 6) Clean up when you're done (destructive!).
# cf.delete_stack(StackName=stack_name)
```



## Pitfalls & quick tips

- Prefer **Change Sets** for updates in production (review before applying).
- Avoid putting **secrets** in templates; use Secrets Manager / SSM Parameter Store.
- Be explicit about delete behavior for data resources (e.g., S3/RDS) via **DeletionPolicy**.
- If you create IAM resources, include the right **Capabilities** (or the deployment will fail).
- Use outputs to publish IDs/ARNs for other stacks/services to consume.

## References
- AWS Docs: CloudFormation (templates, stacks, change sets)
- AWS Docs: CloudFormation Waiters + SDK examples
